In [1]:
my_rdd=sc.textFile("file:\Users\Akshay\Desktop\data.csv")
my_rdd.take(4)

header= my_rdd.first()
my_rdd=my_rdd.filter(lambda x :x!=header)
my_rdd.first()

u'42,50,270900,270944,267,17,44,24220,76,108,1687,1,0,80,0.0498,0.2415,0.1818,0.0047,0.4706,1,1,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,Pastry'

In [2]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LabeledPoint, LogisticRegressionModel
import numpy as np

def parse(line):
    fields = line.split(",")
    X_Min = float(fields[0])
    X_Max = float(fields[1])
    Y_Min = float(fields[2])
    Y_Max = float(fields[3])
    Pixels_Areas=float(fields[4])
    X_Perimeter=float(fields[5])
    Y_Perimeter=float(fields[6])
    Sum_of_Luminosity=float(fields[7])
    Minimum_of_Luminosity=float(fields[8])
    Maximum_of_Luminosity=float(fields[9])
    Length_of_Conveyer=float(fields[10])
    TypeOfSteel_A300=float(fields[11])
    TypeOfSteel_A400=float(fields[12])
    Steel_Plate_Thickness=float(fields[13])
    Edges_Index=float(fields[14])
    Empty_Index=float(fields[15])
    Square_Index=float(fields[16])
    Outside_X_Index=float(fields[17])
    Edges_X_Index=float(fields[18])
    Edges_Y_Index=float(fields[19])
    Outside_Global_Index=float(fields[20])
    LogOfAreas=float(fields[21])
    Log_X_Index=float(fields[22])
    Log_Y_Index=float(fields[23])
    Orientation_Index=float(fields[24])
    Luminosity_Index=float(fields[25])
    SigmoidOfAreas=float(fields[26])


    
    if fields[27] == 'Pastry':
            target = 0
    elif fields[27]=='Z_Scratch':
            target = 1
    elif fields[27]=='K_Scratch':
            target = 2
    elif fields[27]=='Stains':
            target = 3
    elif fields[27]=='Bumps':
            target = 4
    elif fields[27]=='Dirtiness':
            target = 5
    else:
            target = 6
    class_var=target
    
    return LabeledPoint(class_var,(X_Min,X_Max,Y_Min,Y_Max,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,\
                                    Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,\
                                    Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,\
                                    Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas))

#     return LabeledPoint(class_var,(X_Min,X_Max,Y_Min,Y_Max,Pixels_Areas,X_Perimeter,Y_Perimeter))

parsedData=my_rdd.map(parse)

parsedData.take(4)

[LabeledPoint(0.0, [42.0,50.0,270900.0,270944.0,267.0,17.0,44.0,24220.0,76.0,108.0,1687.0,1.0,0.0,80.0,0.0498,0.2415,0.1818,0.0047,0.4706,1.0,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822]),
 LabeledPoint(0.0, [645.0,651.0,2538079.0,2538108.0,108.0,10.0,30.0,11397.0,84.0,123.0,1687.0,1.0,0.0,80.0,0.7647,0.3793,0.2069,0.0036,0.6,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984]),
 LabeledPoint(0.0, [829.0,835.0,1553913.0,1553931.0,71.0,8.0,19.0,7972.0,99.0,125.0,1623.0,1.0,0.0,100.0,0.971,0.3426,0.3333,0.0037,0.75,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.215]),
 LabeledPoint(0.0, [853.0,860.0,369370.0,369415.0,176.0,13.0,45.0,18996.0,99.0,126.0,1353.0,0.0,1.0,290.0,0.7287,0.4413,0.1556,0.0052,0.5385,1.0,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212])]

In [10]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from time import time

training, test = parsedData.randomSplit([0.7, 0.3],11L)


In [11]:
t0 = time()
tree_model = DecisionTree.trainClassifier(training, numClasses=7,categoricalFeaturesInfo={},
                                          impurity='gini', maxDepth=4, maxBins=32)
tt = time() - t0 
                                          

print "Classifier trained in {} seconds".format(round(tt,3))

Classifier trained in 8.425 seconds


In [12]:
predictions = tree_model.predict(test.map(lambda x: x.features))

In [13]:
labelsAndPredictions = test.map(lambda lp: lp.label).zip(predictions)

In [17]:
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(test.count())

In [18]:
print('Test Error = ' + str(testErr))

Test Error = 0.36671686747


In [16]:
print('Learned classification tree model:')
print(tree_model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 23 nodes
  If (feature 4 <= 26.0)
   If (feature 13 <= 99.947205)
    If (feature 0 <= 207.111734)
     Predict: 2.0
    Else (feature 0 > 207.111734)
     Predict: 3.0
   Else (feature 13 > 99.947205)
    If (feature 0 <= 483.0)
     Predict: 6.0
    Else (feature 0 > 483.0)
     Predict: 4.0
  Else (feature 4 > 26.0)
   If (feature 17 <= 0.0526)
    If (feature 10 <= 1359.994044)
     If (feature 11 <= 0.0)
      Predict: 5.0
     Else (feature 11 > 0.0)
      Predict: 1.0
    Else (feature 10 > 1359.994044)
     If (feature 16 <= 0.499806)
      Predict: 0.0
     Else (feature 16 > 0.499806)
      Predict: 4.0
   Else (feature 17 > 0.0526)
    If (feature 0 <= 271.794938)
     If (feature 13 <= 50.0)
      Predict: 2.0
     Else (feature 13 > 50.0)
      Predict: 1.0
    Else (feature 0 > 271.794938)
     If (feature 6 <= 384.13959)
      Predict: 6.0
     Else (feature 6 > 384.13959)
      Predict: 1.0


In [19]:
tree_model.save(sc, "C:/Users/Akshay/Desktop/Fall16/Big Data/Part2/HW2/myDecisionTreeClassificationModel11.txt")

Py4JJavaError: An error occurred while calling o151.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/Akshay/Desktop/Fall16/Big%20Data/Part2/HW2/myDecisionTreeClassificationModel11.txt/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1184)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1161)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1161)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:358)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1161)
	at org

In [14]:
from pyspark.mllib.evaluation import MultilabelMetrics,MulticlassMetrics

In [15]:
metrics=MulticlassMetrics(labelsAndPredictions)
accuracy=metrics.accuracy
#accuracy=(1-testErr)*100
print("Accuracy is :%s"%accuracy)

Accuracy is :0.63328313253


In [20]:
confusion_matrix=metrics.confusionMatrix()
print("Confusion Matrix :%s"%confusion_matrix)

Confusion Matrix :DenseMatrix([[ 128.,    6.,    4.,    0.,   21.,  124.,   54.],
             [   6.,  178.,    0.,    0.,   12.,    0.,    9.],
             [   0.,    0.,  179.,    0.,    0.,    0.,    1.],
             [   0.,    0.,    0.,  152.,    0.,    0.,    0.],
             [  35.,    7.,   19.,    3.,  140.,    4.,   82.],
             [  17.,    1.,    5.,   24.,   17.,   60.,   33.],
             [   0.,    1.,    0.,    0.,    2.,    0.,    4.]])
